# Database

In [ ]:
%config Completer.use_jedi = False

In [ ]:
import pandas as pd
import re

In [ ]:
datas = pd.read_json('./tests/Processor_tests.json').T

In [ ]:
datas.to_csv('./database.csv', sep=';', encoding='latin-1')

# Converter

## CBR

In [ ]:
from comicsreader.converter import cbr2cbz

In [ ]:
cbr2cbz('./tests/01 Wasteland.cbr')

## Pdf

In [ ]:
from comicsreader.converter import pdf2cbz

In [ ]:
pdf2cbz('./tests/Invisible Republic - T01.pdf')

# Dev Meta data

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import pandas as pd
from comicsreader.metaprocessor import MetaProcessor as Meta
from comicsreader.metaprocessor import TitleProcessor as Proc
import re

In [ ]:
path = 'D:/Mes bds/'

In [ ]:
for root, dirs, files in os.walk(path, topdown=False):
    pass

In [ ]:
root

In [ ]:
files

In [ ]:
file = files[3]
file

In [ ]:
pattern = re.compile(r"""
                                (?!{)  # ignore when starting with {
                                ([^A-Za-z][0-9]{1,3})
                                (-[^A-Za-z][0-9]{1,3})?
                                (?!})  # ignore when finishing with }

                                """, re.VERBOSE
                                 )
# pattern = '[0-9]{1,3}'
match = re.search(pattern, file)
match

In [ ]:
match.groups()

In [ ]:
processor = Proc()
processor(file)

In [ ]:
file

In [ ]:
datas = []
for file in files:
    datas.append(Meta.from_file(file))

processed = pd.DataFrame([data.as_dict() for data in datas])

In [ ]:
from ipywidgets import widgets, interactive, interact

pd.set_option("max_colwidth", 100)

@interact(n=list(range(processed.index.stop // 5)))
def sample(n):
    return processed.loc[5*n:5*(n+1), ['file', 'extension', 'date', 'volumes', 'chapters', 'title']]

In [ ]:
import json

In [ ]:
processed.T.to_json('./tests/Processor_tests.json', indent=4)

In [ ]:
processed.to_csv('./database.csv', sep=';', encoding='latin-1')

# Database

In [1]:
import pandas as pd
import re, os
import shutil
import ast
from comicsreader.converter import cbr2cbz, pdf2cbz

%load_ext autoreload
%autoreload 2

In [17]:
def format_list(x, prefix='(', suffix=')'):
    if len(x) == 0:
        result = ''
    elif len(x) < 3:
        result = prefix + '-'.join(map(str, x)) + suffix
    else:
        result = prefix + ', '.join(map(str, x)) + suffix
    return result

def format_title(x):
    dates = format_list(x.dates, '(', ')')
    volumes = format_list(x.volumes, ', T', '')
#     chapters = format_list(x.chapters, ', ', '')
    result = f'{x.title} {dates}{volumes}'
    result = re.sub(' +', ' ', result.strip())
    return result

In [18]:
datas = pd.read_csv('./database_corrected.csv', sep=';', encoding='latin-1', index_col=0)
datas[['dates', 'volumes']] = datas[['dates', 'volumes']].applymap(ast.literal_eval)

In [19]:
datas['formatted_title'] = datas.apply(format_title, axis=1)

In [ ]:
from comicsreader.converter import _clean_directory

In [ ]:
# convert

export_path = 'E:/comics_database/'
input_path = 'D:/Mes bds'

for series, group in datas.groupby('title'):
    path = os.path.join(export_path, series)
    if not os.path.exists(path):
        os.makedirs(path)
    
    for _, row in group.iterrows():
        # convert
        file = row.file
        ext = file.split('.')[-1]
        if ext == 'cbz':
            shutil.copy(os.path.join(input_path, file), os.path.join(path, file))
        elif ext == 'cbr':
            cbr2cbz(os.path.join(input_path, file), path)
        elif ext == 'pdf':
            pdf2cbz(os.path.join(input_path, file), path)
            
        # rename
        input_file = file.split('.')[0] + '.cbz'
        output_file = row.formatted_title + '.cbz'
        os.rename(os.path.join(path, input_file), os.path.join(path, output_file))

In [21]:
series_table = pd.DataFrame(datas.title.unique(), columns=['Serie'])
series_table['ID'] = series_table.Serie.apply(hash)
series_map = series_table.set_index('Serie').ID

books_table = datas.copy()
books_table['serie_ID'] = series_map[books_table['title']].values
books_table.drop(columns=['tokenized_file', 'extension', 'title'], inplace=True)
books_table.rename({'formatted_title': 'name'}, axis=1, inplace=True)
books_table['ID']= books_table.name.apply(hash)
books_table[['dates', 'volumes']] = books_table[['dates', 'volumes']].astype(str)

series_table.rename(str.lower, axis=1, inplace=True)
books_table.rename(str.lower, axis=1, inplace=True)

In [ ]:
import sqlite3
from sqlite3 import Error

def create_connection(path):
    db = None
    try:
        db = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return db

def execute_query(db, query):
    cursor = db.cursor()
    try:
        cursor.execute(query)
        db.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")
        
def query_db(db, query, args=(), one=False):
    cursor = db.cursor()
    cursor.execute(query)
    rv = cursor.fetchall()
    cursor.close()
    return (make_dicts(cursor, rv[0]) if rv else None) if one else tuple([make_dicts(cursor, value) for value in rv])

def make_dicts(cursor, row):
    return dict(((cursor.description[idx][0], value) for idx, value in enumerate(row)))

In [ ]:
series_frame_to_sql = lambda x: f"({x['ID']}, '{x['Series']}')"
books_frame_to_sql = lambda x: f"({x['ID']}, {x['series_ID']}, '{x['name']}', '{x['date']}', '{x['volumes']}')"
frame_to_sql = lambda x, func: ', '.join(x.apply(func, axis=1))

# sqlite

In [ ]:
db = create_connection('./comics_database.sqlite')

In [ ]:
create_series_table_query = """
CREATE TABLE IF NOT EXISTS series (
  id INTEGER PRIMARY KEY,
  series TEXT NOT NULL
);
"""
execute_query(db, create_series_table_query)

In [ ]:
values_to_insert = frame_to_sql(series_table, series_frame_to_sql)

insert_series_values_query = f"""
INSERT INTO
  series (id, series)
VALUES
  {values_to_insert};
"""

execute_query(db, insert_series_values_query)

In [ ]:
query_db(db, 'SELECT * FROM series')

In [ ]:
create_books_table_query = """
CREATE TABLE IF NOT EXISTS books (
  id INTEGER PRIMARY KEY,
  series_id INTEGER NOT NULL,
  name TEXT NOT NULL,
  date TEXT,
  volumes TEXT,
  FOREIGN KEY (series_id) REFERENCES series (id)
);
"""
execute_query(db, create_books_table_query)

In [ ]:
values_to_insert = frame_to_sql(books_table, books_frame_to_sql)

insert_series_values_query = f"""
INSERT INTO
  books (id, series_id, name, date, volumes)
VALUES
  {values_to_insert};
"""

execute_query(db, insert_series_values_query)

In [ ]:
query = """
SELECT * FROM books b
INNER JOIN series s on s.ID = b.series_ID
where s.series like 'Norse%'
"""

query_db(db, query)

In [ ]:
db.close()

# Sqlalchemy

In [52]:
from sqlalchemy import Column, Integer, String, ForeignKey, Table, MetaData
from sqlalchemy.orm import relationship, backref
from sqlalchemy.ext.declarative import declarative_base
from functools import partial

In [90]:
Base = declarative_base()
class AsDict:
    __attr__ = []
    
    def as_dict(self):
        return {key: self.__getattribute__(key) for key in self.__attr__}
    
    @classmethod
    def from_series(cls, x: pd.Series):
        return cls(**x[cls.__attr__].to_dict())
    
    def __repr__(self):
        return self.as_dict().__str__()

class Serie(Base, AsDict):
    __tablename__ = 'serie'
    __attr__ = ['id', 'serie']
    id = Column(Integer, primary_key=True)
    serie = Column(String)
    books = relationship('Book', backref=backref('serie'))    
    
class Book(Base, AsDict):
    __tablename__ = 'book'
    __attr__ = ['id', 'serie_id', 'name', 'dates','volumes']
    id = Column(Integer, primary_key=True)
    serie_id = Column(Integer, ForeignKey('serie.id'))
    name = Column(String)
    dates = Column(String)
    volumes = Column(String)
    
series = Table('serie', )

In [80]:
from sqlalchemy import create_engine, text, insert
from sqlalchemy.orm import sessionmaker

In [9]:
engine = create_engine('sqlite+pysqlite:///comics_database.sqlite', echo=True, future=True)
Session = sessionmaker(bind=engine)
session = Session()

In [28]:
Base.metadata.create_all(engine)

2021-08-13 12:23:34,886 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-08-13 12:23:34,892 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("serie")
2021-08-13 12:23:34,896 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-08-13 12:23:34,898 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("serie")
2021-08-13 12:23:34,898 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-08-13 12:23:34,898 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("book")
2021-08-13 12:23:34,906 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-08-13 12:23:34,907 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("book")
2021-08-13 12:23:34,907 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-08-13 12:23:34,913 INFO sqlalchemy.engine.Engine 
CREATE TABLE serie (
	id INTEGER NOT NULL, 
	serie VARCHAR, 
	PRIMARY KEY (id)
)


2021-08-13 12:23:34,915 INFO sqlalchemy.engine.Engine [no key 0.00147s] ()
2021-08-13 12:23:34,924 INFO sqlalchemy.engine.Engine 
CREATE TABLE book (
	id INTEGER NOT NULL, 
	serie_i

In [ ]:
def execute_query(engine, query, **args):
    with engine.connect() as conn:
        conn.execute(query, *args)
        conn.commit()

In [ ]:
session.add_all()

In [95]:
values_to_insert = [Serie.from_series(row) for _, row in series_table.iterrows()]
session.add_all(values_to_insert)
session.commit()

In [98]:
values_to_insert = [Book.from_series(row) for _, row in books_table.iterrows()]
session.add_all(values_to_insert)
session.commit()

2021-08-13 18:13:28,896 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-08-13 18:13:28,912 INFO sqlalchemy.engine.Engine INSERT INTO book (id, serie_id, name, dates, volumes) VALUES (?, ?, ?, ?, ?)
2021-08-13 18:13:28,912 INFO sqlalchemy.engine.Engine [generated in 0.00362s] ((1360680357879757109, 1360680357879757109, 'Authority - Human on the Inside', '[]', '[]'), (4145634991153928280, -7681845462247089994, 'Blackgas (2006)', '[2006]', '[]'), (9010358171587930173, 2124202763486797453, 'Caliban (2014)', '[2014]', '[]'), (-6127913855428277606, 490953623514464766, 'Critical Role - Vox Machina Origins Series II (2020), T5', '[2020]', '[5]'), (-8684704682130249962, 490953623514464766, 'Critical Role - Vox Machina Origins Series II (2019), T1', '[2019]', '[1]'), (448587910115620161, 490953623514464766, 'Critical Role - Vox Machina Origins Series II (2019), T2', '[2019]', '[2]'), (4135652741296550171, 490953623514464766, 'Critical Role - Vox Machina Origins Series II (2019), T3', '[2019]

In [99]:
session.close()

# pyunpack

In [ ]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

from pyunpack import Archive
from zipfile import ZipFile
import os

# Rarfile

In [ ]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

import rarfile.rarfile as rarfile

# rarfile.UNAR_TOOL = './unar/unar.exe'
# rarfile.UNAR_TOOL = './unrar/UNRAR.exe'
rarfile.UNAR_TOOL = './libarchive/bin/bsdtar.exe'

In [ ]:
file_path = './01 Wasteland.cbr'
with rarfile.RarFile(file_path) as rf:
    for f in rf.infolist():
        print(f.filename, f.file_size)
        rf.extract(f.orig_filename, './')

In [ ]:
f.orig_filename.endswith(b'/')